# **Cinematica inversa de un brazo robotico**

La posicion del extremo del brazo en el plano esta dada por

$$
x(\theta_1,\theta_2) = L_1 \cos(\theta_1) + L_2 \cos(\theta_1 + \theta_2) \\
y(\theta_1,\theta_2) = L_1 \sin(\theta_1) + L_2 \sin(\theta_1 + \theta_2)
$$


<center>

![alt text](./images/brazo-robotico.png)
</center>

Para alcanzar un punto deseado $(x_d, y_d)$ se plantea el siguiente sistema

$$
F_1(\theta_1,\theta_2) = x(\theta_1,\theta_2) - x_d = 0 \\
F_2(\theta_1,\theta_2) = y(\theta_1,\theta_2) - y_d = 0
$$